In [0]:
import requests
import os
from zipfile import ZipFile
from io import BytesIO


In [0]:
# Diretório no DBFS
dbfs_path = "/FileStore/combustiveis/"
dbutils.fs.mkdirs(dbfs_path)

True

In [0]:
# Base URL
base_url = "https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/"

# Lista de arquivos
arquivos = [
    ("ca-2020-01.csv", "csv"), ("ca-2020-02.csv", "csv"),
    ("ca-2021-01.csv", "csv"), ("ca-2021-02.csv", "csv"),
    ("precos-semestrais-ca.zip", "zip"), ("ca-2022-02.zip", "zip"),
    ("ca-2023-01.zip", "zip"), ("ca-2023-02.zip", "zip"),
    ("ca-2024-01.zip", "zip"), ("ca-2024-02.zip", "zip")
]

In [0]:
# Diretório no DBFS
dbfs_path = "/FileStore/combustiveis/"
dbutils.fs.mkdirs(dbfs_path)

# Função para verificar se o arquivo existe no DBFS
def file_exists(dbfs_file_path):
    try:
        dbutils.fs.ls(dbfs_file_path)
        return True
    except:
        return False

# Baixar e salvar
for arquivo, formato in arquivos:
    # Definir nome do arquivo destino (ca-AAAA-SS.csv)
    if formato == "csv":
        new_file_name = arquivo
    else:
        year_semester = arquivo.replace("ca-", "").replace(".zip", "")
        new_file_name = f"ca-{year_semester}.csv"
    
    dbfs_file_path = f"{dbfs_path}{new_file_name}"
    
    # Verificar se o arquivo já existe
    if file_exists(dbfs_file_path):
        print(f"Arquivo {dbfs_file_path} já existe, pulando...")
        continue
    
    # Baixar o arquivo
    url = base_url + arquivo
    print(f"Baixando {url}...")
    response = requests.get(url)
    
    if response.status_code == 200:
        if formato == "csv":
            # Salvar CSV diretamente no DBFS
            with open(f"/tmp/{new_file_name}", "wb") as f:
                f.write(response.content)
            dbutils.fs.cp(f"file:/tmp/{new_file_name}", dbfs_file_path)
            print(f"Salvo: {dbfs_file_path}")
        else:
            # Extrair CSV do ZIP e salvar como ca-AAAA-SS.csv
            try:
                with ZipFile(BytesIO(response.content)) as zip_file:
                    for file_name in zip_file.namelist():
                        if file_name.endswith(".csv"):
                            csv_content = zip_file.read(file_name)
                            with open(f"/tmp/{new_file_name}", "wb") as f:
                                f.write(csv_content)
                            dbutils.fs.cp(f"file:/tmp/{new_file_name}", dbfs_file_path)
                            print(f"Salvo: {dbfs_file_path}")
                            break
            except Exception as e:
                print(f"Erro ao processar {arquivo}: {str(e)}")
    else:
        print(f"Erro ao baixar {arquivo}: {response.status_code}")

Arquivo /FileStore/combustiveis/ca-2020-01.csv já existe, pulando...
Arquivo /FileStore/combustiveis/ca-2020-02.csv já existe, pulando...
Arquivo /FileStore/combustiveis/ca-2021-01.csv já existe, pulando...
Arquivo /FileStore/combustiveis/ca-2021-02.csv já existe, pulando...
Baixando https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/precos-semestrais-ca.zip...
Salvo: /FileStore/combustiveis/ca-precos-semestrais-ca.csv
Baixando https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/ca-2022-02.zip...
Salvo: /FileStore/combustiveis/ca-2022-02.csv
Baixando https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/ca-2023-01.zip...
Salvo: /FileStore/combustiveis/ca-2023-01.csv
Baixando https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/ca-2023-02.zip...
Salvo: /FileStore/combustiveis/ca-2023-02.csv
Baixando https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-

In [0]:
# Listar arquivos no DBFS
display(dbutils.fs.ls(dbfs_path))

path,name,size,modificationTime
dbfs:/FileStore/combustiveis/ca-2020-01.csv,ca-2020-01.csv,86655383,1744491611000
dbfs:/FileStore/combustiveis/ca-2020-02.csv,ca-2020-02.csv,38903171,1744491679000
dbfs:/FileStore/combustiveis/ca-2021-01.csv,ca-2021-01.csv,57911124,1744491778000
dbfs:/FileStore/combustiveis/ca-2021-02.csv,ca-2021-02.csv,81317769,1744491919000
dbfs:/FileStore/combustiveis/ca-2022-02.csv,ca-2022-02.csv,64921361,1744492637000
dbfs:/FileStore/combustiveis/ca-2023-01.csv,ca-2023-01.csv,74291202,1744492658000
dbfs:/FileStore/combustiveis/ca-2023-02.csv,ca-2023-02.csv,80630173,1744492680000
dbfs:/FileStore/combustiveis/ca-2024-01.csv,ca-2024-01.csv,81365972,1744492702000
dbfs:/FileStore/combustiveis/ca-2024-02.csv,ca-2024-02.csv,71797479,1744492721000
dbfs:/FileStore/combustiveis/ca-precos-semestrais-ca.csv,ca-precos-semestrais-ca.csv,90812404,1744492619000
